In [1]:
# ============================================================
# Cell 1: Imports (only)
# ============================================================

using LinearAlgebra
using Statistics
using Random
using Printf

using Distributions
using FastGaussQuadrature

using SparseArrays
using IterativeSolvers

using Interpolations
using Parameters

In [2]:
# ============================================================
# Cell 2: Structs (only) — mutable, keyword
# ============================================================

@with_kw mutable struct Params
    # Preferences / time
    r::Float64

    # Aggregate CTMC
    Q::Matrix{Float64}                 # K×K generator
    zgrid::Vector{Float64}             # length K (labels or levels, optional)

    # Matching (unskilled)
    μU::Float64
    ηU::Float64
    kU::Float64                        # vacancy posting cost
    βU::Float64                        # bargaining weight
    λU::Float64                        # redraw intensity

    # Training technology
    ϕ::Float64                         # completion rate
    # cost function parameters for c(x) will be handled by a separate object

    # Sector productivity shifters (unskilled)
    PU::Vector{Float64}                # length K, PU(z)

    # Flow payoffs (unskilled, training) by aggregate state
    bU::Vector{Float64}                # length K
    bT::Vector{Float64}                # length K

    # Distributions: Beta parameters
    ax::Float64
    bx::Float64
    ap::Float64
    bp::Float64
end


@with_kw mutable struct Grids
    x::Vector{Float64}
    p::Vector{Float64}
    z_idx::Vector{Int}                 # 1:K convenience
    K::Int
    Nx::Int
    Np::Int
end


@with_kw mutable struct BetaCDFCache
    # Precomputed objects for fast integrals over Beta densities on [0,1]
    # Typical usage: integrate f(p) dG(p) via weights over bins or quadrature nodes.

    # For p-distribution (unconditional Beta(ap,bp) or conditional on x if you later generalize)
    p_cdf::Vector{Float64}             # CDF evaluated on p-grid
    p_pdf::Vector{Float64}             # PDF evaluated on p-grid (optional)

    # For x-distribution (Beta(ax,bx)) if you need expectations over x
    x_cdf::Vector{Float64}
    x_pdf::Vector{Float64}

    # Integration weights for p-grid (e.g., bin probabilities)
    p_weights::Vector{Float64}         # length Np-1 or Np depending on convention
    x_weights::Vector{Float64}         # same idea
end


@with_kw mutable struct CostSpec
    # Training cost c(x) is model-specific. Keep it parameterized.
    # Example option: c(x) = c0 + c1*(1-x)^κ (convex and decreasing in x)
    c0::Float64
    c1::Float64
    κ::Float64
end


@with_kw mutable struct Objects
    # Endogenous equilibrium objects by aggregate state z and worker type x / productivity p.
    # Dimensions:
    #   U_search[x,z], T[x,z] are Nx×K
    #   U[x,z] is Nx×K
    #   J[p,x,z] is Np×Nx×K
    #   pstar[x,z] is Nx×K
    #   xbar[z] is K

    θ::Vector{Float64}                 # length K
    xbar::Vector{Float64}              # length K

    U_search::Matrix{Float64}          # Nx×K
    T::Matrix{Float64}                 # Nx×K
    U::Matrix{Float64}                 # Nx×K

    J::Array{Float64,3}                # Np×Nx×K
    pstar::Matrix{Float64}             # Nx×K
end


@with_kw mutable struct SolverSettings
    max_iter_outer::Int = 500
    max_iter_inner::Int = 2000

    tol_outer::Float64 = 1e-8
    tol_inner::Float64 = 1e-10

    damp_θ::Float64 = 0.3              # damping for θ updates
    damp_pstar::Float64 = 0.5          # damping for p* updates
    damp_xbar::Float64 = 0.5           # damping for xbar updates
    damp_J::Float64 = 0.5              # damping for J updates

    debug_wage::Bool = false          
    verbose::Bool = true
end


@with_kw mutable struct Model
    par::Params
    grid::Grids
    cache::BetaCDFCache
    cost::CostSpec
    obj::Objects
    set::SolverSettings
end

Model

In [3]:
# ============================================================
# Cell 3: Core utilities (functions only)
# ============================================================

"""
    apply_RminusQ!(out, V, par)

Compute (r - Q) * V in-place.

- If `V` is a vector of length K, `out` is a vector length K.
- If `V` is a matrix of size (N×K), `out` is a matrix (N×K) and the operator is applied
  across the z-dimension (columns), i.e. row-by-row: for each i, out[i,:] = (r - Q)*V[i,:].

Assumes `par.Q` is K×K and `par.r` is scalar.
"""
function apply_RminusQ!(out::AbstractVector, V::AbstractVector, par::Params)
    K = size(par.Q, 1)
    @assert length(V) == K "V must have length K = size(Q,1)."
    @assert length(out) == K "out must have length K."

    r = par.r
    Q = par.Q

    # out = r*V - Q*V
    mul!(out, Q, V)               # out = Q*V
    @inbounds for k in 1:K
        out[k] = r*V[k] - out[k]
    end
    return nothing
end

function apply_RminusQ!(out::AbstractMatrix, V::AbstractMatrix, par::Params)
    K = size(par.Q, 1)
    @assert size(V, 2) == K "V must have K columns (z dimension)."
    @assert size(out) == size(V) "out must match size(V)."

    r = par.r
    Q = par.Q
    N = size(V, 1)

    # For each row i: out[i,:] = r*V[i,:] - (V[i,:] * Q')'
    # Equivalent to: out = r*V - V*Q'
    mul!(out, V, transpose(Q))     # out = V*Q'
    @inbounds for i in 1:N, k in 1:K
        out[i,k] = r*V[i,k] - out[i,k]
    end
    return nothing
end

"""
    apply_RminusQ(V, par) -> similar(V)

Allocating version of `apply_RminusQ!`.
"""
function apply_RminusQ(V::AbstractVector, par::Params)
    out = similar(V)
    apply_RminusQ!(out, V, par)
    return out
end

function apply_RminusQ(V::AbstractMatrix, par::Params)
    out = similar(V)
    apply_RminusQ!(out, V, par)
    return out
end


"""
    q_U(θ, par) -> Float64

Vacancy-filling rate q_U(θ) = μU * θ^{-ηU}. Requires θ>0.
"""
@inline function q_U(θ::Real, par::Params)::Float64
    @assert θ > 0 "θ must be positive."
    return par.μU * (float(θ))^(-par.ηU)
end

"""
    f_U(θ, par) -> Float64

Job-finding rate f(θ) = θ*q_U(θ) = μU * θ^{1-ηU}. Requires θ>0.
"""
@inline function f_U(θ::Real, par::Params)::Float64
    @assert θ > 0 "θ must be positive."
    return par.μU * (float(θ))^(1 - par.ηU)
end


"""
    c(x, cost) -> Float64

Training cost c(x). Default parameterization:
    c(x) = c0 + c1*(1 - x)^κ

This is decreasing in x for c1>0, κ>0 and convex for κ>=1 (on [0,1]).
"""
@inline function c(x::Real, cost::CostSpec)::Float64
    xx = float(x)
    @assert 0.0 <= xx <= 1.0 "x must lie in [0,1]."
    return cost.c0 + cost.c1 * (1 - xx)^(cost.κ)
end


# ----------------------------
# Beta CDF/PDF cache + weights
# ----------------------------

"""
    build_beta_cache(grid, par) -> BetaCDFCache

Precompute Beta CDF/PDF on x- and p-grids, and build integration weights from CDF differences.

Weights convention (bin probabilities):
- For a grid u[1] < ... < u[N], define midpoints:
    m[1] = u[1]
    m[i] = (u[i-1] + u[i]) / 2 for i=2..N
    m[N+1] = u[N]
- Then define weights w[i] = F(m[i+1]) - F(m[i]) for i=1..N.

These weights sum (approximately) to 1 and support fast approximations:
    ∫ f(u) dF(u) ≈ Σ_i f(u[i]) * w[i]
"""
function build_beta_cache(grid::Grids, par::Params)::BetaCDFCache
    @assert all(diff(grid.x) .> 0) "x grid must be strictly increasing."
    @assert all(diff(grid.p) .> 0) "p grid must be strictly increasing."

    Dx = Beta(par.ax, par.bx)
    Dp = Beta(par.ap, par.bp)

    x = grid.x
    p = grid.p
    Nx = length(x)
    Np = length(p)

    x_cdf = cdf.(Ref(Dx), x)
    x_pdf = pdf.(Ref(Dx), x)

    p_cdf = cdf.(Ref(Dp), p)
    p_pdf = pdf.(Ref(Dp), p)

    # helper: bin weights from CDF on a grid
    function bin_weights_from_cdf(u::Vector{Float64}, D::Distribution)
        N = length(u)
        mids = Vector{Float64}(undef, N + 1)
        mids[1] = u[1]
        @inbounds for i in 2:N
            mids[i] = 0.5 * (u[i-1] + u[i])
        end
        mids[N+1] = u[N]

        Fm = cdf.(Ref(D), mids)
        w = Vector{Float64}(undef, N)
        @inbounds for i in 1:N
            w[i] = max(Fm[i+1] - Fm[i], 0.0)  # guard tiny negatives from floating error
        end

        s = sum(w)
        if s > 0
            @inbounds for i in 1:N
                w[i] /= s
            end
        end
        return w
    end

    x_weights = bin_weights_from_cdf(Vector{Float64}(x), Dx)
    p_weights = bin_weights_from_cdf(Vector{Float64}(p), Dp)

    return BetaCDFCache(
        p_cdf = p_cdf,
        p_pdf = p_pdf,
        x_cdf = x_cdf,
        x_pdf = x_pdf,
        p_weights = p_weights,
        x_weights = x_weights
    )
end


"""
    find_cutoff_index_p(pgrid, pstar) -> Int

Return the smallest index i such that pgrid[i] >= pstar.
If pstar <= pgrid[1], returns 1. If pstar > pgrid[end], returns length(pgrid)+1
(convention: "empty integral").
"""
function find_cutoff_index_p(pgrid::AbstractVector{<:Real}, pstar::Real)::Int
    Np = length(pgrid)
    ps = float(pstar)

    if ps <= pgrid[1]
        return 1
    elseif ps > pgrid[end]
        return Np + 1
    else
        # searchsortedfirst returns insertion position to keep sorted order
        return searchsortedfirst(pgrid, ps)
    end
end


"""
    integrate_over_p(vals_p, idx0, cache) -> Float64

Approximate ∫_{p*}^1 vals(p) dG(p) using precomputed bin weights on the p-grid.

Inputs:
- vals_p: length Np vector with values at p-grid nodes
- idx0: cutoff index as returned by `find_cutoff_index_p`:
    * idx0 == Np+1 means p* above the grid => integral is 0
- cache.p_weights: length Np weights summing to 1

Output:
- scalar approximation of the truncated expectation.
"""
function integrate_over_p(vals_p::AbstractVector{<:Real}, idx0::Int, cache::BetaCDFCache)::Float64
    Np = length(cache.p_weights)
    @assert length(vals_p) == Np "vals_p must have length Np."
    if idx0 >= Np + 1
        return 0.0
    end
    s = 0.0
    @inbounds for i in idx0:Np
        s += float(vals_p[i]) * cache.p_weights[i]
    end
    return s
end


"""
    integrate_over_x(vals_x, idx0, cache) -> Float64

Analogous utility for expectations over x:
    ∫_{x*}^1 vals(x) dL(x) ≈ Σ_{i>=idx0} vals_x[i]*x_weights[i].

Uses the same cutoff-index convention as `find_cutoff_index_p` (but for x-grids).
"""
function integrate_over_x(vals_x::AbstractVector{<:Real}, idx0::Int, cache::BetaCDFCache)::Float64
    Nx = length(cache.x_weights)
    @assert length(vals_x) == Nx "vals_x must have length Nx."
    if idx0 >= Nx + 1
        return 0.0
    end
    s = 0.0
    @inbounds for i in idx0:Nx
        s += float(vals_x[i]) * cache.x_weights[i]
    end
    return s
end

integrate_over_x

In [4]:
# ============================================================
# Cell 4: Distribution / integration utilities
# ============================================================

# ----------------------------
# 1) Generic grid weight builder
# ----------------------------

"""
    bin_weights_from_cdf(u, D) -> w

Construct discrete weights on a monotone grid `u` using CDF differences at cell midpoints.

For u[1] < ... < u[N], define midpoints:
    m[1]   = u[1]
    m[i]   = (u[i-1] + u[i]) / 2   for i = 2..N
    m[N+1] = u[N]

Then weights:
    w[i] = F(m[i+1]) - F(m[i])     for i = 1..N

We normalize weights to sum to 1 (guarding against tiny floating error).
Interpretation:
    ∫ f(u) dF(u) ≈ Σ_i f(u[i]) * w[i]

Inputs:
- u::Vector{Float64}: strictly increasing grid in [0,1]
- D::Distribution: supports cdf(D, x)

Output:
- w::Vector{Float64} length N, nonnegative, sums to 1
"""
function bin_weights_from_cdf(u::Vector{Float64}, D::Distribution)::Vector{Float64}
    N = length(u)
    @assert N >= 2 "Need at least 2 grid points."
    @assert all(diff(u) .> 0) "Grid must be strictly increasing."
    @assert u[1] >= 0.0 - 1e-12 && u[end] <= 1.0 + 1e-12 "Grid should lie in [0,1]."

    mids = Vector{Float64}(undef, N + 1)
    mids[1] = u[1]
    @inbounds for i in 2:N
        mids[i] = 0.5 * (u[i-1] + u[i])
    end
    mids[N+1] = u[end]

    Fm = cdf.(Ref(D), mids)
    w  = Vector{Float64}(undef, N)
    @inbounds for i in 1:N
        w[i] = max(Fm[i+1] - Fm[i], 0.0)  # clip tiny negatives from floating error
    end

    s = sum(w)
    @assert s > 0 "Weight sum is zero; check distribution support / grid."
    @inbounds for i in 1:N
        w[i] /= s
    end
    return w
end


# ----------------------------
# 2) Beta cache builder (x, p, y)
# ----------------------------

"""
    build_beta_cache(grid, par; ay=nothing, by=nothing, ygrid=nothing) -> BetaCDFCache

Build CDF/PDF arrays and bin-weights for x and p on the provided grids, using Beta parameters in `par`.

If you later want y as well, you can either:
- extend Params to include (ay,by) and extend BetaCDFCache to include y_* fields, or
- build a separate cache for y.

This function currently uses:
- x ~ Beta(par.ax, par.bx) on grid.x
- p ~ Beta(par.ap, par.bp) on grid.p

Returns:
- BetaCDFCache populated with x/p cdf/pdf and weights.
"""
function build_beta_cache(grid::Grids, par::Params)::BetaCDFCache
    x = Vector{Float64}(grid.x)
    p = Vector{Float64}(grid.p)

    Dx = Beta(par.ax, par.bx)
    Dp = Beta(par.ap, par.bp)

    x_cdf = cdf.(Ref(Dx), x)
    x_pdf = pdf.(Ref(Dx), x)
    p_cdf = cdf.(Ref(Dp), p)
    p_pdf = pdf.(Ref(Dp), p)

    x_w = bin_weights_from_cdf(x, Dx)
    p_w = bin_weights_from_cdf(p, Dp)

    return BetaCDFCache(
        p_cdf = p_cdf,
        p_pdf = p_pdf,
        x_cdf = x_cdf,
        x_pdf = x_pdf,
        p_weights = p_w,
        x_weights = x_w
    )
end


# ----------------------------
# 3) Cutoff bracketing and indices
# ----------------------------

"""
    cutoff_index(grid, ustar) -> Int

Generic cutoff index utility for any increasing grid.

Convention:
- returns the smallest i such that grid[i] >= ustar
- if ustar <= grid[1]      => returns 1
- if ustar >  grid[end]    => returns N+1 (means "empty upper tail")
"""
function cutoff_index(grid::AbstractVector{<:Real}, ustar::Real)::Int
    N = length(grid)
    us = float(ustar)
    if us <= grid[1]
        return 1
    elseif us > grid[end]
        return N + 1
    else
        return searchsortedfirst(grid, us)
    end
end


"""
    bracket_root_on_grid(grid, vals) -> Union{Nothing, Tuple{Int,Int}}

Given `vals[i]` defined on monotone grid `grid[i]`, find a pair (iL,iH) such that:
    vals[iL] <= 0 <= vals[iH]
with iH = iL+1.

Returns:
- (iL,iH) if a sign change exists
- nothing otherwise

Use case: finding p* where J(p*,x,z)=0.
"""
function bracket_root_on_grid(vals::AbstractVector{<:Real})::Union{Nothing,Tuple{Int,Int}}
    N = length(vals)
    @inbounds for i in 1:(N-1)
        if (vals[i] <= 0.0 && vals[i+1] >= 0.0) || (vals[i] >= 0.0 && vals[i+1] <= 0.0)
            return (i, i+1)
        end
    end
    return nothing
end


"""
    linear_root(grid, vals, iL, iH) -> Float64

Linear interpolation root between indices iL<iH (typically iH=iL+1), assuming
vals[iL] and vals[iH] have opposite signs or one is zero.

Returns u* in [grid[iL], grid[iH]].
"""
function linear_root(grid::AbstractVector{<:Real}, vals::AbstractVector{<:Real}, iL::Int, iH::Int)::Float64
    uL = float(grid[iL]); uH = float(grid[iH])
    vL = float(vals[iL]); vH = float(vals[iH])

    if abs(vH - vL) < 1e-14
        return 0.5*(uL + uH)
    end
    # u* = uL + (0 - vL) * (uH-uL)/(vH-vL)
    return uL - vL * (uH - uL) / (vH - vL)
end


# ----------------------------
# 4) Fast truncated expectations using precomputed weights
# ----------------------------

"""
    tail_expectation(vals, w, idx0) -> Float64

Compute Σ_{i=idx0..N} vals[i]*w[i]. If idx0==N+1 => 0.
"""
@inline function tail_expectation(vals::AbstractVector{<:Real}, w::AbstractVector{<:Real}, idx0::Int)::Float64
    N = length(w)
    @assert length(vals) == N "vals and weights must have the same length."
    if idx0 >= N + 1
        return 0.0
    end
    s = 0.0
    @inbounds for i in idx0:N
        s += float(vals[i]) * float(w[i])
    end
    return s
end


"""
    tail_mass(w, idx0) -> Float64

Compute Σ_{i=idx0..N} w[i]. If idx0==N+1 => 0.
Useful for acceptance probabilities.
"""
@inline function tail_mass(w::AbstractVector{<:Real}, idx0::Int)::Float64
    N = length(w)
    if idx0 >= N + 1
        return 0.0
    end
    s = 0.0
    @inbounds for i in idx0:N
        s += float(w[i])
    end
    return s
end


"""
    truncated_mean(vals, w, idx0) -> Float64

Compute conditional expectation:
    E[vals | u >= u*] ≈ (Σ_{i>=idx0} vals[i]w[i]) / (Σ_{i>=idx0} w[i])

Returns 0.0 if tail mass is zero.
"""
function truncated_mean(vals::AbstractVector{<:Real}, w::AbstractVector{<:Real}, idx0::Int)::Float64
    mass = tail_mass(w, idx0)
    if mass <= 0.0
        return 0.0
    end
    return tail_expectation(vals, w, idx0) / mass
end


# ----------------------------
# 5) Composition over x: searchers-only weights
# ----------------------------

"""
    searcher_weights_x(grid, cache, xbar_value) -> Vector{Float64}

Construct a probability mass function over x-grid restricted to "searchers",
given cutoff xbar(z): search iff x < xbar.

We return normalized weights over x nodes:
    wS[i] ∝ x_weights[i]  for x[i] < xbar
    wS[i] = 0 otherwise

If no mass remains (e.g., xbar <= x[1]), returns all zeros.
"""
function searcher_weights_x(grid::Grids, cache::BetaCDFCache, xbar_value::Real)::Vector{Float64}
    x = grid.x
    Nx = length(x)
    wS = zeros(Float64, Nx)
    xb = float(xbar_value)

    @inbounds for i in 1:Nx
        if x[i] < xb
            wS[i] = cache.x_weights[i]
        end
    end
    s = sum(wS)
    if s > 0
        @inbounds for i in 1:Nx
            wS[i] /= s
        end
    end
    return wS
end


"""
    expected_over_searchers_x(vals_x, wS) -> Float64

Compute Σ_i vals_x[i] * wS[i] where wS is already normalized over searchers.
"""
@inline function expected_over_searchers_x(vals_x::AbstractVector{<:Real}, wS::AbstractVector{<:Real})::Float64
    @assert length(vals_x) == length(wS)
    s = 0.0
    @inbounds for i in eachindex(vals_x)
        s += float(vals_x[i]) * float(wS[i])
    end
    return s
end

expected_over_searchers_x

In [5]:
# ============================================================
# Cell 5: initialize_model (and helpers used only for init)
# ============================================================

"""
    default_grid(N; lo=1e-6, hi=1-1e-6) -> Vector{Float64}

Convenience grid on [0,1] avoiding endpoints (safe for Beta pdf/cdf).
"""
function default_grid(N::Int; lo::Float64=1e-6, hi::Float64=1-1e-6)::Vector{Float64}
    @assert N >= 2
    @assert 0.0 < lo < hi < 1.0
    return collect(range(lo, hi; length=N))
end


"""
    alloc_objects(grid, par; θ0=nothing, pstar0=nothing, xbar0=nothing) -> Objects

Allocate and initialize all endogenous objects with reasonable default guesses.
"""
function alloc_objects(grid::Grids, par::Params, cost::CostSpec;
                       θ0=nothing, pstar0=nothing, xbar0=nothing)::Objects
    Nx, Np, K = grid.Nx, grid.Np, grid.K

    θ    = θ0    === nothing ? fill(1.0, K) : Vector{Float64}(θ0)
    xbar = xbar0 === nothing ? fill(1.0, K) : Vector{Float64}(xbar0)

    # pstar initial: start with low reservation so most matches accepted
    pstar = pstar0 === nothing ? fill(grid.p[1], Nx, K) : Matrix{Float64}(pstar0)

    # Initial unemployment/search values: bU/r as crude baseline (row-by-row)
    U_search = zeros(Float64, Nx, K)
    @inbounds for z in 1:K, ix in 1:Nx
        U_search[ix,z] = par.bU[z] / par.r
    end

    # Initial training value: solve in steady-ish way ignoring Q: T ≈ (bT + ϕ U_S)/(r+ϕ)
    # We set U_S=0 placeholder here; you can overwrite later.
    T = zeros(Float64, Nx, K)
    @inbounds for z in 1:K, ix in 1:Nx
        T[ix,z] = par.bT[z] / (par.r + par.ϕ)
    end

    # Construct U via cutoff rule (xbar initially 1 => almost all search)
    U = similar(U_search)
    @inbounds for z in 1:K, ix in 1:Nx
        if grid.x[ix] < xbar[z]
            U[ix,z] = U_search[ix,z]
        else
            U[ix,z] = -c(grid.x[ix], cost) + T[ix,z]
        end
    end

    # Firm value J: initialize at zero
    J = zeros(Float64, Np, Nx, K)

    return Objects(θ=θ, xbar=xbar, U_search=U_search, T=T, U=U, J=J, pstar=pstar)
end


"""
    initialize_model(; kwargs...) -> Model

Create Params/Grids/Cache/Cost/Settings/Objects, fully initialized.

Required keywords (no defaults):
- Q::Matrix{Float64}        (K×K generator)
- zgrid::Vector{Float64}    (length K)
- PU::Vector{Float64}       (length K)
- bU::Vector{Float64}       (length K)
- bT::Vector{Float64}       (length K)

All other parameters have defaults (edit as you like).
"""
function initialize_model(;
    # --- Required aggregate objects ---
    Q::Matrix{Float64},
    zgrid::Vector{Float64},
    PU::Vector{Float64},
    bU::Vector{Float64},
    bT::Vector{Float64},

    # --- Core parameters (defaults) ---
    r::Float64 = 0.05,

    μU::Float64 = 1.0,
    ηU::Float64 = 0.5,
    kU::Float64 = 0.5,
    βU::Float64 = 0.5,
    λU::Float64 = 0.2,

    ϕ::Float64 = 0.1,

    # --- Beta parameters (defaults; can be changed) ---
    ax::Float64 = 2.0, bx::Float64 = 2.0,
    ap::Float64 = 2.0, bp::Float64 = 2.0,

    # --- Grids ---
    Nx::Int = 41,
    Np::Int = 61,
    xgrid::Union{Nothing,Vector{Float64}} = nothing,
    pgrid::Union{Nothing,Vector{Float64}} = nothing,

    # --- Training cost parameters ---
    c0::Float64 = 0.0,
    c1::Float64 = 1.0,
    κ::Float64  = 2.0,

    # --- Solver settings ---
    max_iter_outer::Int = 500,
    max_iter_inner::Int = 2000,
    tol_outer::Float64  = 1e-8,
    tol_inner::Float64  = 1e-10,
    damp_θ::Float64     = 0.3,
    damp_pstar::Float64 = 0.5,
    damp_xbar::Float64  = 0.5,
    damp_J::Float64     = 0.5,
    debug_wage::Bool    = false,
    verbose::Bool       = true,

    # --- Optional initial guesses ---
    θ0 = nothing,
    pstar0 = nothing,
    xbar0 = nothing
)
    K = length(zgrid)
    @assert size(Q,1) == K && size(Q,2) == K "Q must be K×K with K=length(zgrid)."
    @assert length(PU) == K && length(bU) == K && length(bT) == K "PU,bU,bT must be length K."

    x = xgrid === nothing ? default_grid(Nx) : xgrid
    p = pgrid === nothing ? default_grid(Np) : pgrid

    grid = Grids(x=x, p=p, z_idx=collect(1:K), K=K, Nx=length(x), Np=length(p))

    par = Params(
        r=r, Q=Q, zgrid=zgrid,
        μU=μU, ηU=ηU, kU=kU, βU=βU, λU=λU,
        ϕ=ϕ,
        PU=PU, bU=bU, bT=bT,
        ax=ax, bx=bx, ap=ap, bp=bp
    )

    cost = CostSpec(c0=c0, c1=c1, κ=κ)

    set = SolverSettings(
        max_iter_outer=max_iter_outer,
        max_iter_inner=max_iter_inner,
        tol_outer=tol_outer,
        tol_inner=tol_inner,
        damp_θ=damp_θ,
        damp_pstar=damp_pstar,
        damp_xbar=damp_xbar,
        verbose=verbose
    )

    cache = build_beta_cache(grid, par)

    obj = alloc_objects(grid, par, cost; θ0=θ0, pstar0=pstar0, xbar0=xbar0)

    return Model(par=par, grid=grid, cache=cache, cost=cost, obj=obj, set=set)
end

initialize_model

In [6]:
# ============================================================
# Cell 6: Inner-loop functions (given θ, update J, p*, U_search, T, xbar, U)
# ============================================================

# ----------------------------
# Wage helper (placeholder)
# ----------------------------

"""
    wage_U(p, Ux_z, PU_z, par) -> Float64

Baseline wage mapping used inside the unskilled inner loop.

IMPORTANT:
This is a placeholder that enforces a Nash-style split in a simple way:
    w = β * (PU*p) + (1-β) * (r * U)

This is convenient for a stable first implementation and can be replaced later
once you decide on the exact wage equation implied by your preferred bargaining
protocol with redraw shocks.
"""
@inline function wage_U(p::Real, Ux_z::Real, PU_z::Real, par::Params)::Float64
    return par.βU * (PU_z * float(p)) + (1 - par.βU) * (par.r * float(Ux_z))
end


# ----------------------------
# 1) Solve firm values J given current θ and current U
# ----------------------------

"""
    solve_J!(model) -> nothing

Inner fixed point for firm values J(p,x,z), given current U(x,z).

We iterate on J using:
  A_J * J_new(p,x,:) = RHS(p,x,:) + λU * Epos(x,:)

where
  A_J = ((r+λU)I - Q) is K×K and constant across (p,x),
  RHS(p,x,z) = PU(z)*p - w(p,x,z),
  Epos(x,z) = E_p[max(0, J_old(p',x,z))] under Beta weights.

This implements the expectation operator max{0,J} appearing in your firm Bellman equation.
"""
function solve_J!(model::Model)
    par, grid, cache, obj, set = model.par, model.grid, model.cache, model.obj, model.set
    Nx, Np, K = grid.Nx, grid.Np, grid.K

    # (r + λ)I - Q
    A = (par.r + par.λU) * I(K) - par.Q
    F = lu(Matrix(A))  # factor once per call

    # Work arrays
    J_old = copy(obj.J)
    J_rhs = similar(obj.J)     # raw (undamped) new J
    J_tmp = similar(obj.J)     # damped update
    rhs   = zeros(Float64, K)
    Epos  = zeros(Float64, Nx, K)

    # Sanity: weights should integrate to ~1
    # (comment out once confirmed)
    # @assert abs(sum(cache.p_weights) - 1.0) < 1e-6

    dampJ = set.damp_J

    for it in 1:set.max_iter_inner

        # 1) Epos(x,z) = ∫ max{0,J_old(p',x,z)} dG(p')
        @inbounds for ix in 1:Nx, z in 1:K
            s = 0.0
            for ip in 1:Np
                s += max(0.0, J_old[ip,ix,z]) * cache.p_weights[ip]
            end
            Epos[ix,z] = s
        end

        # 2) compute raw J_rhs by solving K-dim system for each (p,x)
        @inbounds for ix in 1:Nx
            Ux = view(obj.U, ix, :)
            for ip in 1:Np
                pval = grid.p[ip]
                for z in 1:K
                    # wage
                    w = if set.debug_wage
                        # Debug wage (decouples from U): flow Nash on output vs bU
                        # You may need par.bU available; adjust if your Params differs.
                        par.bU[z] + par.βU * (par.PU[z]*pval - par.bU[z])
                    else
                        wage_U(pval, Ux[z], par.PU[z], par)
                    end

                    # Feasibility guard: keep w strictly below output to avoid pathological J
                    y = par.PU[z] * pval
                    w = min(w, y - 1e-10)

                    rhs[z] = y - w + par.λU * Epos[ix,z]
                end

                sol = F \ rhs
                for z in 1:K
                    J_rhs[ip,ix,z] = sol[z]
                end
            end
        end

        # 3) Damp the operator and check convergence on the damped update
        @. J_tmp = (1 - dampJ) * J_old + dampJ * J_rhs
        err = maximum(abs.(J_tmp .- J_old))

        # Commit
        obj.J .= J_tmp

        if err < set.tol_inner
            return nothing
        end

        # Continue
        J_old .= J_tmp
    end
    return nothing
end


# ----------------------------
# 2) Update reservation productivity p*(x,z) from J(p*,x,z)=0
# ----------------------------

"""
    update_pstar!(model) -> nothing

Compute pstar(x,z) by locating the zero of J(p,x,z) on the p-grid.
If J is always positive => p* = p_min (accept all).
If J is always negative => p* = p_max + tiny (reject all, integral tail empty).
"""
function update_pstar!(model::Model)
    grid, obj, set = model.grid, model.obj, model.set
    Nx, Np, K = grid.Nx, grid.Np, grid.K
    p = grid.p

    pstar_new = similar(obj.pstar)

    @inbounds for z in 1:K, ix in 1:Nx
        Jvec = view(obj.J, :, ix, z)

        # Cases
        if all(Jvec .>= 0.0)
            pstar_new[ix,z] = p[1]
        elseif all(Jvec .<= 0.0)
            pstar_new[ix,z] = p[end] + 1e-6
        else
            br = bracket_root_on_grid(Jvec)
            if br === nothing
                # fallback: choose smallest p with J>=0
                idx = findfirst(x -> x >= 0.0, Jvec)
                pstar_new[ix,z] = idx === nothing ? (p[end] + 1e-6) : p[idx]
            else
                iL, iH = br
                pstar_new[ix,z] = linear_root(p, Jvec, iL, iH)
            end
        end
    end

    # Damping
    obj.pstar .= (1 - set.damp_pstar) .* obj.pstar .+ set.damp_pstar .* pstar_new
    return nothing
end


# ----------------------------
# 3) Update U_search using expected firm value over acceptable draws
# ----------------------------

"""
    update_U_search!(model) -> nothing

For each x, solve the K-dim linear system:
    (rI - Q) U_search[x,:] = bU + f(z)*β/(1-β)*∫_{p*}^1 J(p,x,z) dG(p)

Uses Beta bin weights for dG and current θ(z), pstar(x,z), J.
"""
function update_U_search!(model::Model)
    par, grid, cache, obj = model.par, model.grid, model.cache, model.obj
    Nx, Np, K = grid.Nx, grid.Np, grid.K

    A = par.r * I(K) - par.Q
    F = lu(Matrix(A))

    rhs = zeros(Float64, K)
    Unew = similar(obj.U_search)

    @inbounds for ix in 1:Nx
        for z in 1:K
            idx0 = cutoff_index(grid.p, obj.pstar[ix,z])
            Jvec = view(obj.J, :, ix, z)
            EJ  = tail_expectation(Jvec, cache.p_weights, idx0)  # ∫_{p*}^1 J dG
            rhs[z] = par.bU[z] + f_U(obj.θ[z], par) * (par.βU/(1-par.βU)) * EJ
        end
        sol = F \ rhs
        for z in 1:K
            Unew[ix,z] = sol[z]
        end
    end

    obj.U_search .= Unew
    return nothing
end


# ----------------------------
# 4) Update training value T
# ----------------------------

"""
    update_T!(model; U_S=nothing) -> nothing

Solve:
    (rI - Q)T = bT + ϕ(U_S - T)
equivalently:
    ((r+ϕ)I - Q) T = bT + ϕ U_S

If `U_S` is not provided, we set U_S = 0 (placeholder).
"""
function update_T!(model::Model; U_S::Union{Nothing,Matrix{Float64}}=nothing)
    par, grid, obj = model.par, model.grid, model.obj
    Nx, K = grid.Nx, grid.K

    US = U_S === nothing ? zeros(Float64, Nx, K) : U_S
    @assert size(US) == (Nx, K)

    A = (par.r + par.ϕ) * I(K) - par.Q
    F = lu(Matrix(A))

    rhs = zeros(Float64, K)
    Tnew = similar(obj.T)

    @inbounds for ix in 1:Nx
        for z in 1:K
            rhs[z] = par.bT[z] + par.ϕ * US[ix,z]
        end
        sol = F \ rhs
        for z in 1:K
            Tnew[ix,z] = sol[z]
        end
    end

    obj.T .= Tnew
    return nothing
end


# ----------------------------
# 5) Update xbar(z) and construct U(x,z) by cutoff rule
# ----------------------------

"""
    update_xbar_and_U!(model) -> nothing

Define:
  V_train(x,z) = -c(x) + T(x,z)
Training optimal iff V_train >= U_search.

We compute xbar(z) as the smallest x at which training becomes optimal.
Then set:
  U(x,z) = U_search(x,z) if x < xbar(z)
         = V_train(x,z)  otherwise
"""
function update_xbar_and_U!(model::Model)
    grid, obj, cost, set = model.grid, model.obj, model.cost, model.set
    Nx, K = grid.Nx, grid.K
    x = grid.x

    xbar_new = similar(obj.xbar)
    Unew = similar(obj.U)

    @inbounds for z in 1:K
        # diff(x) = U_search - V_train; training is optimal when diff <= 0
        diff = Vector{Float64}(undef, Nx)
        for ix in 1:Nx
            Vtrain = -c(x[ix], cost) + obj.T[ix,z]
            diff[ix] = obj.U_search[ix,z] - Vtrain
        end

        if all(diff .<= 0.0)
            # everyone trains
            xbar_new[z] = 0.0
        elseif all(diff .> 0.0)
            # no one trains
            xbar_new[z] = 1.0
        else
            # locate first crossing to training-optimal region
            # i.e. smallest ix such that diff[ix] <= 0
            ix0 = findfirst(d -> d <= 0.0, diff)
            if ix0 === nothing
                xbar_new[z] = 1.0
            elseif ix0 == 1
                xbar_new[z] = 0.0
            else
                # linear interpolation between ix0-1 (diff>0) and ix0 (diff<=0)
                iL, iH = ix0-1, ix0
                xbar_new[z] = linear_root(x, diff, iL, iH)
                xbar_new[z] = clamp(xbar_new[z], 0.0, 1.0)
            end
        end

        # construct U by cutoff
        xb = xbar_new[z]
        for ix in 1:Nx
            if x[ix] < xb
                Unew[ix,z] = obj.U_search[ix,z]
            else
                Unew[ix,z] = -c(x[ix], cost) + obj.T[ix,z]
            end
        end
    end

    # Damping
    obj.xbar .= (1 - set.damp_xbar) .* obj.xbar .+ set.damp_xbar .* xbar_new
    obj.U .= Unew
    return nothing
end


# ----------------------------
# 6) Inner loop driver (fixed θ): iterate until p*, U_search, T, xbar, U, J converge
# ----------------------------

"""
    inner_loop!(model; U_S=nothing) -> nothing

Given current θ(z) (held fixed), iterate:
  J -> p* -> U_search -> T -> xbar,U
until convergence.

This is the "inner loop" of your algorithm.
"""
function inner_loop!(model::Model; U_S::Union{Nothing,Matrix{Float64}}=nothing)
    set, obj = model.set, model.obj

    for it in 1:set.max_iter_inner
        # Save for convergence checks
        θ_old     = copy(obj.θ)       # θ is fixed here; kept for completeness
        p_old     = copy(obj.pstar)
        xbar_old  = copy(obj.xbar)
        U_old     = copy(obj.U)
        J_old     = copy(obj.J)

        solve_J!(model)
        update_pstar!(model)
        update_U_search!(model)
        update_T!(model; U_S=U_S)
        update_xbar_and_U!(model)

        errJ    = maximum(abs.(obj.J .- J_old))
        errp    = maximum(abs.(obj.pstar .- p_old))
        errxbar = maximum(abs.(obj.xbar .- xbar_old))
        errU    = maximum(abs.(obj.U .- U_old))

        err = maximum((errJ, errp, errxbar, errU))

        if set.verbose && (it % 25 == 0)
            @printf("  inner it=%4d | errJ=%.3e errp=%.3e errxbar=%.3e errU=%.3e\n",
                    it, errJ, errp, errxbar, errU)
        end

        if err < set.tol_inner
            if set.verbose
                @printf("  inner converged at it=%d with err=%.3e\n", it, err)
            end
            return nothing
        end
    end

    if set.verbose
        @printf("  inner hit max_iter_inner=%d\n", set.max_iter_inner)
    end
    return nothing
end

inner_loop!

In [7]:
# ============================================================
# Cell 7: Outer loop + solve_model!
# ============================================================

# ----------------------------
# 1) Job creation residual (per z)
# ----------------------------

"""
    job_creation_rhs_z(z, model) -> Float64

Compute the RHS term in free entry for a given aggregate state z:

    RHS(z) = q(θ(z)) * E_x_searchers [ ∫_{p*(x,z)}^1 J(p,x,z) dG(p) ]

where the expectation over x is taken over SEARCHERS only (x < xbar(z)),
using normalized searcher weights.

Returns RHS(z). Free entry condition is:
    kU = RHS(z)
"""
function job_creation_rhs_z(z::Int, model::Model)::Float64
    par, grid, cache, obj = model.par, model.grid, model.cache, model.obj
    Nx, Np = grid.Nx, grid.Np

    θz = obj.θ[z]
    qz = q_U(θz, par)

    # weights over x restricted to searchers (x < xbar(z))
    wS = searcher_weights_x(grid, cache, obj.xbar[z])
    if sum(wS) == 0.0
        return 0.0
    end

    # for each x: compute EJ(x,z) = ∫_{p*(x,z)}^1 J(p,x,z) dG(p)
    EJx = zeros(Float64, Nx)
    @inbounds for ix in 1:Nx
        idx0 = cutoff_index(grid.p, obj.pstar[ix,z])
        Jvec = view(obj.J, :, ix, z)
        EJx[ix] = tail_expectation(Jvec, cache.p_weights, idx0)
    end

    Ex = expected_over_searchers_x(EJx, wS)
    return qz * Ex
end


"""
    job_creation_residual(θ, z, model) -> Float64

Residual for free entry at state z:
    res(θ) = kU - q(θ) * E_x_searchers [ ∫_{p*(x,z)}^1 J(p,x,z) dG(p) ]

IMPORTANT:
This residual depends on θ both directly (through q(θ)) and indirectly because J and p*
depend on θ. In the outer loop, we treat the indirect dependence via fixed-point iteration:
we update θ using the current inner-loop solution objects.
"""
function job_creation_residual(θ::Real, z::Int, model::Model)::Float64
    # Temporarily evaluate residual using the *current* EJ expectation,
    # but scaling it with q(θ) at the candidate θ.
    par, grid, cache, obj = model.par, model.grid, model.cache, model.obj

    # Searcher weights at current xbar(z)
    wS = searcher_weights_x(grid, cache, obj.xbar[z])
    if sum(wS) == 0.0
        return par.kU  # RHS=0 => residual=kU
    end

    Nx = grid.Nx
    EJx = zeros(Float64, Nx)
    @inbounds for ix in 1:Nx
        idx0 = cutoff_index(grid.p, obj.pstar[ix,z])
        Jvec = view(obj.J, :, ix, z)
        EJx[ix] = tail_expectation(Jvec, cache.p_weights, idx0)
    end
    Ex = expected_over_searchers_x(EJx, wS)

    return par.kU - q_U(θ, par) * Ex
end


# ----------------------------
# 2) Robust θ update per z (bisection)
# ----------------------------

"""
    bracket_theta_for_bisection(z, model; θmin=1e-6, θmax=1e6, growth=10.0, max_expand=40)
        -> (θL, θH)

Find θL < θH such that job_creation_residual(θL) and job_creation_residual(θH)
have opposite signs (or one is ~0).

If no bracket is found, returns the last interval.
"""
function bracket_theta_for_bisection(z::Int, model::Model;
                                     θmin::Float64=1e-6,
                                     θmax::Float64=1e6,
                                     growth::Float64=10.0,
                                     max_expand::Int=40)

    θL = θmin
    θH = min(θmin * growth, θmax)

    rL = job_creation_residual(θL, z, model)
    rH = job_creation_residual(θH, z, model)

    # expand until sign change or bounds hit
    it = 0
    while (sign(rL) == sign(rH)) && (θH < θmax) && (it < max_expand)
        θL = θH
        rL = rH
        θH = min(θH * growth, θmax)
        rH = job_creation_residual(θH, z, model)
        it += 1
    end
    return (θL, θH)
end


"""
    solve_theta_z(z, model; tol=1e-10, maxit=200) -> Float64

Solve job_creation_residual(θ,z)=0 using bisection on a bracket found by
`bracket_theta_for_bisection`.

If residual does not bracket a root (e.g., always positive), returns boundary value:
- if residual>0 everywhere tested: return θmax (tightness must rise to increase RHS)
- if residual<0 everywhere tested: return θmin
"""
function solve_theta_z(z::Int, model::Model;
                       tol::Float64=1e-10,
                       maxit::Int=200,
                       θmin::Float64=1e-6,
                       θmax::Float64=1e6)

    θL, θH = bracket_theta_for_bisection(z, model; θmin=θmin, θmax=θmax)
    rL = job_creation_residual(θL, z, model)
    rH = job_creation_residual(θH, z, model)

    # If no sign change, choose boundary based on direction
    if sign(rL) == sign(rH)
        # residual = kU - q(θ)*Ex; if residual>0 -> RHS too small -> increase θ (decrease q though!).
        # NOTE: Because q decreases in θ, the direction is opposite of standard job-finding intuition.
        # Here, job creation uses q(θ) not f(θ). With q decreasing, if RHS is too small we need LOWER θ.
        # So:
        #   residual>0 => need bigger RHS => need higher q => need smaller θ.
        #   residual<0 => RHS too big => need lower q => need larger θ.
        return rL > 0 ? θmin : θmax
    end

    # Standard bisection
    for _ in 1:maxit
        θM = 0.5*(θL + θH)
        rM = job_creation_residual(θM, z, model)
        if abs(rM) < tol || (θH - θL) < tol
            return θM
        end
        if sign(rM) == sign(rL)
            θL = θM
            rL = rM
        else
            θH = θM
            rH = rM
        end
    end

    return 0.5*(θL + θH)
end


"""
    update_theta!(model; θmin=1e-6, θmax=1e6) -> nothing

Update θ(z) state-by-state using bisection solutions, then apply damping.
"""
function update_theta!(model::Model; θmin::Float64=1e-6, θmax::Float64=1e6)
    grid, obj, set = model.grid, model.obj, model.set
    K = grid.K

    θ_new = similar(obj.θ)

    @inbounds for z in 1:K
        θ_new[z] = solve_theta_z(z, model; θmin=θmin, θmax=θmax)
        θ_new[z] = max(θ_new[z], θmin)
    end

    obj.θ .= (1 - set.damp_θ) .* obj.θ .+ set.damp_θ .* θ_new
    return nothing
end


# ----------------------------
# 3) Convergence diagnostics
# ----------------------------

"""
    supnorm(a, b) -> Float64

Maximum absolute difference between arrays of the same size.
"""
@inline function supnorm(a, b)::Float64
    @assert size(a) == size(b)
    return maximum(abs.(a .- b))
end


"""
    outer_errors(old_obj, obj) -> NamedTuple

Compute sup-norm errors for key equilibrium objects.
"""
function outer_errors(old_obj::Objects, obj::Objects)
    return (
        errθ     = supnorm(old_obj.θ, obj.θ),
        errpstar = supnorm(old_obj.pstar, obj.pstar),
        errxbar  = supnorm(old_obj.xbar, obj.xbar),
        errU     = supnorm(old_obj.U, obj.U),
        errJ     = supnorm(old_obj.J, obj.J)
    )
end


"""
    print_outer_progress(it, errs)

Pretty-print outer loop progress.
"""
function print_outer_progress(it::Int, errs)
    @printf("outer it=%4d | errθ=%.3e errp=%.3e errxbar=%.3e errU=%.3e errJ=%.3e\n",
            it, errs.errθ, errs.errpstar, errs.errxbar, errs.errU, errs.errJ)
end


# ----------------------------
# 4) solve_model! (outer loop driver)
# ----------------------------

"""
    solve_model!(model; U_S=nothing) -> Model

Run the full nested fixed-point algorithm:

Outer loop over θ(z):
  - given θ, run inner_loop! to convergence (updates J, p*, U_search, T, xbar, U)
  - update θ from job creation
until joint convergence of (θ, p*, xbar, U, J).

Returns the model (mutated in place).
"""
function solve_model!(model::Model; U_S::Union{Nothing,Matrix{Float64}}=nothing)
    set, obj = model.set, model.obj

    for it in 1:set.max_iter_outer
        old = deepcopy(obj)

        # Inner loop: solve conditional on current θ
        inner_loop!(model; U_S=U_S)

        # Outer update: θ(z) from job creation
        update_theta!(model)

        # Convergence diagnostics
        errs = outer_errors(old, obj)
        err = maximum((errs.errθ, errs.errpstar, errs.errxbar, errs.errU, errs.errJ))

        if set.verbose
            print_outer_progress(it, errs)
        end

        if err < set.tol_outer
            if set.verbose
                @printf("outer converged at it=%d with err=%.3e\n", it, err)
            end
            return model
        end
    end

    if set.verbose
        @printf("outer hit max_iter_outer=%d\n", set.max_iter_outer)
    end
    return model
end

solve_model!

In [8]:
# ============================================================
# Cell 8: Smoke test / example run (choose sensible numbers)
# ============================================================

# Reproducibility
Random.seed!(1234)

# --- Aggregate CTMC (K=3 states: low/normal/high) ---
zgrid = [0.75, 1.0 , 1.25]  # labels only (not used directly in equations)

# Generator: switch at intensity 0.10 in either direction
Q = [-0.10  0.10  0.00;
      0.10 -0.20  0.10;
      0.00  0.10 -0.10]

# Sector productivity shifter in unskilled market by state
PU = [0.75, 1.0, 1.25]

# Flow payoffs by state
bU = [0.30, 0.35, 0.40]   # unemployment benefit / home production
bT = [0.25, 0.30, 0.35]   # training flow payoff (lower than unemp is fine)
# --- Initialize model ---
m = initialize_model(
    Q=Q, zgrid=zgrid, PU=PU, bU=bU, bT=bT,

    # Discounting
    r=0.05,

    # Matching (unskilled)
    μU=0.9,
    ηU=0.5,
    kU=0.25,
    βU=0.5,
    λU=0.30,   # redraw intensity

    # Training completion
    ϕ=0.15,

    # Beta distributions on [0,1]
    ax=2.0, bx=2.0,   # x ~ Beta(2,2)
    ap=2.5, bp=2.0,   # p ~ Beta(2.5,2)

    # Grids
    Nx=31,
    Np=51,

    # Training cost: c(x) = c0 + c1*(1-x)^κ
    c0=0.05,
    c1=0.60,
    κ=2.0,

    # Solver settings (a bit loose for smoke test; tighten later)
    max_iter_outer=200,
    max_iter_inner=25,
    tol_outer=1e-7,
    tol_inner=1e-9,
    damp_θ=0.9,
    damp_pstar=0.9,
    damp_xbar=0.9,
    damp_J=0.7,
    verbose=true
)

# --- Solve ---
solve_model!(m)

# --- Report key outcomes ---
println("\nConverged objects (unskilled market):")
println("θ(z):   ", m.obj.θ)
println("xbar(z):", m.obj.xbar)

# Show a few p*(x,z) values
ixs = [1, Int(round(m.grid.Nx/2)), m.grid.Nx]
for z in 1:m.grid.K
    println("\nState z=$z:")
    for ix in ixs
        @printf("  x=%.3f  p*(x,z)=%.4f\n", m.grid.x[ix], m.obj.pstar[ix,z])
    end
end

# Check free entry residuals at the converged θ
println("\nFree entry residual check (should be near 0):")
for z in 1:m.grid.K
    res = job_creation_residual(m.obj.θ[z], z, m)
    @printf("  z=%d  residual=%.3e\n", z, res)
end

  inner it=  25 | errJ=1.902e+00 errp=0.000e+00 errxbar=0.000e+00 errU=1.820e+01
  inner hit max_iter_inner=25
outer it=   1 | errθ=2.150e+01 errp=0.000e+00 errxbar=0.000e+00 errU=2.058e+01 errJ=2.073e+00
  inner it=  25 | errJ=1.868e+00 errp=0.000e+00 errxbar=0.000e+00 errU=7.601e+01
  inner hit max_iter_inner=25
outer it=   2 | errθ=1.990e+01 errp=0.000e+00 errxbar=0.000e+00 errU=8.781e+00 errJ=1.901e+00
  inner it=  25 | errJ=1.690e+00 errp=0.000e+00 errxbar=0.000e+00 errU=2.239e+01
  inner hit max_iter_inner=25
outer it=   3 | errθ=2.093e+00 errp=0.000e+00 errxbar=0.000e+00 errU=8.829e+00 errJ=2.788e-02
  inner it=  25 | errJ=2.022e+00 errp=0.000e+00 errxbar=0.000e+00 errU=1.421e+01
  inner hit max_iter_inner=25
outer it=   4 | errθ=2.881e+01 errp=0.000e+00 errxbar=0.000e+00 errU=1.297e+01 errJ=2.141e+00
  inner it=  25 | errJ=2.007e+00 errp=0.000e+00 errxbar=0.000e+00 errU=9.715e+01
  inner hit max_iter_inner=25
outer it=   5 | errθ=2.596e+01 errp=0.000e+00 errxbar=0.000e+00 errU=